## Summary
This notebook use a pre-trained transformer model (arabic-bert-base) for Named Entity Recognition (NER) in Arabic. The model was trained on a corpus of 14,000 sentences (378,000 tokens) and can recognize entities such as person, location, organization, etc. You can find more details about the model and the corpus [here](https://huggingface.co/hatmimoha/arabic-ner). The notebook also shows how to preprocess the input text, split long text into shorter segments, and clean the output. Finally, it shows how to test the model on random text and deploy it using FastAPI.

In [1]:
#Load pre-train model for Arabic Named Entity Recognition
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
model = AutoModelForTokenClassification.from_pretrained("hatmimoha/arabic-ner")
tokenizer = AutoTokenizer.from_pretrained("hatmimoha/arabic-ner")

In [48]:
# Test with data sample

sequence="© Reuters. اسهم كوكا كولا تتراجع بنسبة 5% في الربع الثاني Investing.com – تراجعت اسهم كوكا خلال التداولات قبل طرحها في الأسواق بعد ان ذكرت سلسلة مطاعم الوجبات السريعة وجود ارباح أفضل من المتوقع في الربع الثاني يوم الأربعاء، فيما غابت التوقعات حول الإيرادات. و على وجه التحديد، ذكرت أكبر شركة مشروبات في العالم ان أرباح للسهم الواحد سجلت 0.60 دولار. وتراجعت الايرادات في الفترة من نيسان/أبريل إلى حزيران/يونيو بنسبة 5.0٪ ليصل إلى 11.45 بليون دولار. وكان المحللون قد توقعوا ارباح بمقدار 0.58 دولار لتصل العائدات الى عائدات من 11.76 بليون دولار. فيما يتعلق بالتوقعات، تتوقع شركة كوكا كولا ان ترتفع الايرادات العضوية بنسبة 3٪، بانخفاض عن التوقعات السابقة لها بنسبة 4٪ لتصل إلى 5٪. وأضافت الشركة أن ربحية السهم مقارنة للسنة كاملة كان من المتوقع أن تنخفض 4٪ إلى 7٪ مقابل مقارنة ب 2.00 دولار في السنة السابقة. ويترقب المستثمرون مؤتمر الشركة الساعة 09:00 بالتوقيت الشرقي، أو 13: 0(بتوقيت جرينتش). وبعد صدور التقرير تراجعت اسهم بنسبة 1.27٪ لتصل إلى 44.31 دولار في السوق بانخفاض عن سعر اغلاق يوم الثلاثاء م البالغ 44.88 دولار ."

## Preprocessing
This step cleans the text by removing punctuation, English characters and numbers. It also segments the text into sub tokens and merges them to form words.

In [3]:
import re


def remove_punctuation(text):
    """Remove punctuation, English chars and numbers from text."""
    pattern = r"[^\w\s]|[\d]|[a-zA-Z]+"
    text = re.sub(pattern, "", text)
    return text


In [50]:
remove_punctuation(sequence)

'  اسهم كوكا كولا تتراجع بنسبة  في الربع الثاني   تراجعت اسهم كوكا خلال التداولات قبل طرحها في الأسواق بعد ان ذكرت سلسلة مطاعم الوجبات السريعة وجود ارباح أفضل من المتوقع في الربع الثاني يوم الأربعاء فيما غابت التوقعات حول الإيرادات و على وجه التحديد ذكرت أكبر شركة مشروبات في العالم ان أرباح للسهم الواحد سجلت  دولار وتراجعت الايرادات في الفترة من نيسانأبريل إلى حزيرانيونيو بنسبة  ليصل إلى  بليون دولار وكان المحللون قد توقعوا ارباح بمقدار  دولار لتصل العائدات الى عائدات من  بليون دولار فيما يتعلق بالتوقعات تتوقع شركة كوكا كولا ان ترتفع الايرادات العضوية بنسبة  بانخفاض عن التوقعات السابقة لها بنسبة  لتصل إلى  وأضافت الشركة أن ربحية السهم مقارنة للسنة كاملة كان من المتوقع أن تنخفض  إلى  مقابل مقارنة ب  دولار في السنة السابقة ويترقب المستثمرون مؤتمر الشركة الساعة  بالتوقيت الشرقي أو  بتوقيت جرينتش وبعد صدور التقرير تراجعت اسهم بنسبة  لتصل إلى  دولار في السوق بانخفاض عن سعر اغلاق يوم الثلاثاء م البالغ  دولار '

In [4]:
#To deal with long sequence
def split_text(text, max_length=512):
    """Split text into smaller chunks."""
    chunks = []
    start = 0
    text = remove_punctuation(text)
    while start < len(text):
        end = start + max_length
        if end >= len(text):
            chunks.append(text[start:])
            break
        else:
            end = text.rfind(" ", start, end)
            if end == -1:
                end = start + max_length
            chunks.append(text[start:end])
            start = end + 1
    return chunks

In [56]:
#test split_text
len(split_text(sequence))
k = split_text(sequence)[0]

In [57]:
# Bit of a hack to get the tokens with the special tokens
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(k)))
inputs = tokenizer.encode(k, return_tensors="pt")
outputs = model(inputs).logits
predictions = torch.argmax(outputs, dim=2)

In [58]:
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

('[CLS]', 'O')
('اسهم', 'O')
('كوك', 'B-ORGANIZATION')
('##ا', 'B-ORGANIZATION')
('كول', 'I-ORGANIZATION')
('##ا', 'I-ORGANIZATION')
('تت', 'O')
('##راجع', 'O')
('بنسبة', 'O')
('في', 'O')
('الربع', 'O')
('الثاني', 'O')
('تراجعت', 'O')
('اسهم', 'O')
('كوك', 'B-ORGANIZATION')
('##ا', 'B-ORGANIZATION')
('خلال', 'O')
('التداول', 'O')
('##ات', 'O')
('قبل', 'O')
('طرحها', 'O')
('في', 'O')
('الاسواق', 'O')
('بعد', 'O')
('ان', 'O')
('ذكرت', 'O')
('سلسلة', 'B-ORGANIZATION')
('مطاعم', 'O')
('الوجبات', 'I-ORGANIZATION')
('السريعة', 'I-ORGANIZATION')
('وجود', 'O')
('ارباح', 'O')
('افضل', 'O')
('من', 'O')
('المتوقع', 'O')
('في', 'O')
('الربع', 'O')
('الثاني', 'O')
('يوم', 'B-DATE')
('الاربعاء', 'I-DATE')
('فيما', 'O')
('غاب', 'O')
('##ت', 'O')
('التوقعات', 'O')
('حول', 'O')
('الايرادات', 'O')
('و', 'O')
('على', 'O')
('وجه', 'O')
('التحديد', 'O')
('ذكرت', 'O')
('اكبر', 'O')
('شركة', 'O')
('مشروبات', 'O')
('في', 'O')
('العالم', 'O')
('ان', 'O')
('ارباح', 'O')
('للس', 'O')
('##هم', 'O')
('الواحد', 'O'

## Clean the output

In [59]:
def get_ents(tokens, predictions):
  """
  Get only the 3 entities ORGANIZATION,LOCATION and PERSON,
  fix word tokens sub split like this ##ك to get readable words
  """
  org=[]
  loc=[]
  man=[]

  for token, prediction,index in zip(tokens, predictions[0].numpy(),list(range(len(tokens)))):
    if model.config.id2label[prediction].find('ORGANIZATION') != -1:
        if (token.find('##')!=-1) and len(org)>0and (org[-1][1] == index-1) :
            org[-1]= [org[-1][0]+token[2:],index]
        elif token.find('##')==-1:
            org.append([token,index])

    elif model.config.id2label[prediction].find('LOCATION') != -1:
        if (token.find('##')!=-1) and len(loc)>0 and (loc[-1][1] == index-1) :
            loc[-1]= [loc[-1][0]+token[2:],index]
        elif token.find('##')==-1:
            loc.append([token,index])

    elif model.config.id2label[prediction].find('PERSON') != -1:
        if (token.find('##')!=-1) and len(man)>0 and (man[-1][1] == index-1) :
            man[-1]= [man[-1][0]+token[2:],index]
        elif token.find('##')==-1:
            man.append([token,index])
  
  return [ i[0]for i in org],[ i[0]for i in loc],[ i[0]for i in man]


In [60]:
def get_pred(sequence):
  """The function call get_ents function and get tokens and predictions"""
  # Bit of a hack to get the tokens with the special tokens  
  tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
  inputs = tokenizer.encode(sequence, return_tensors="pt")
  outputs = model(inputs).logits
  predictions = torch.argmax(outputs, dim=2)
  return get_ents(tokens, predictions)
  

In [61]:
def func(chunks):
  """This the main predict each chunk and combine the result and return dict"""
  """Important for deploying step"""


  s1 = set()
  s2 = set()
  s3 = set()

  for sequence in chunks:
     org,loc,man = get_pred(sequence)
     s1.update(org)
     s2.update(loc)
     s3.update(man)


  return {'Persons':list(s3),'Organizations':list(s1) , 'Locations':list(s2)}
  


In [62]:
#Test the output
func(split_text(sequence))

{'Persons': [],
 'Organizations': ['كوكا',
  'الوجبات',
  'سلسلة',
  'حزيرانيونيو',
  'كولا',
  'السريعة'],
 'Locations': []}

Sound good let's try random sample

In [68]:
import os
import random
path = "stockNews/"
file = random.choices(os.listdir('stockNews'), k=1)[0]
print("file name : ",file)
sequence = open(path+file).read().replace('\n', '')


func(split_text(sequence))

file name :  28.txt


{'Persons': ['وليام',
  'المنعم',
  'هيل',
  'رشدي',
  'عبد',
  'ايفو',
  'اير',
  'درار',
  'علاء'],
 'Organizations': ['معهد',
  'الاوروبي',
  'شل',
  'داكس',
  'وتوتال',
  'اير',
  'الفرنسي',
  'سيب',
  'المنزلية',
  'الفرنسية',
  'ووليام',
  'فايننشال',
  'ريان',
  'وليام',
  'رويترز',
  'وايني',
  'داتش',
  'للاجهزة',
  'رويال',
  'تايمز',
  'الاتحاد',
  'كاك',
  'ايفو',
  'لرويترز'],
 'Locations': ['لندن', 'اوروبا']}

## Deploying
Load model from Hugging face and using FastAPI library
![alt text](Postman.png "postman")
![alt text](API_Swagger_UI.png "Swagger_UI")

Also use the predefined functions in this notebook